In [3]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import sys
from scipy.interpolate import griddata
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colorbar
from matplotlib import rc
from scipy.optimize import fsolve
from scipy.integrate import simpson
from numpy import trapz
from numpy.polynomial.polynomial import polyfit
from scipy import integrate
from scipy.interpolate import CubicSpline
from scipy.interpolate import UnivariateSpline
from matplotlib.pyplot import cm
from brokenaxes import brokenaxes

plt.rcParams["font.family"] = "Times New Roman"

mpl.rcParams['axes.linewidth'] = 2

In [4]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [5]:
def entropy_contour_plots(Text_dir_entropy,Graph_dir_entropy,N,u,beta):
    
    filename_entropy = "%s/Text_files_N_%s_U_%s/Thermodynamic_entropy_N_%s_U_%s_beta_%s.dat"%(Text_dir_entropy,N,u,N,u,beta[0])
    nden,ent,ent_std = np.loadtxt(filename_entropy,unpack = 'True',usecols = [1,3,4])
        
    Ent = np.zeros(len(nden)*(len(beta)))      #dkappa_dT.tolist()
    Ent_std = np.zeros(len(nden)*(len(beta)))  #dkappa_dT_std.tolist()
    T_val = np.zeros(len(nden)*(len(beta)))      #T.tolist()
    Nden = np.zeros(len(nden)*(len(beta)))      #[float(U[0])*x for x in [1]*len(Trot)]
    n_len = len(nden)
    
    max_den = np.zeros(len(beta))
    Temp = np.zeros(len(beta))
    nfit = np.arange(1.0,1.5,0.005)
    for k in range(len(beta)):
        #print("Trot",Trot[k])
        start_ind = k*n_len
        end_ind = (k+1)*n_len
        filename_entropy = "%s/Text_files_N_%s_U_%s/Thermodynamic_entropy_N_%s_U_%s_beta_%s.dat"%(Text_dir_entropy,N,u,N,u,beta[k])
        num_den,ent,ent_std = np.loadtxt(filename_entropy,unpack = 'True',usecols = [1,3,4])
        Nden[start_ind:end_ind] = np.copy(num_den)
        Ent[start_ind:end_ind] = -1*np.copy(ent)
        Ent_std[start_ind:end_ind] = np.copy(ent_std)
        T_val[start_ind:end_ind] = (1/float(beta[k]))*np.ones(len(nden))
        #hf_ind = find_nearest(num_den,1.0)
        #n_ind = find_nearest(num_den,1.5)
        #cs = CubicSpline(num_den[hf_ind:n_ind],-1*s_k[hf_ind:n_ind])
        #S_fit = cs(nfit)
        #idx = np.argmin(S_fit)     
        #print("hf_end,n_end",hf_ind,num_den[hf_ind],n_ind,num_den[n_ind])
        #print(s_k[hf_ind:n_ind])
        #max_den[k] = nfit[idx]
        Temp[k] = 1/float(beta[k])
        
    print("max_den",max_den)
    N_grid = np.linspace(np.nanmin(Nden),np.nanmax(Nden),num = 2*n_len)
    T_grid = np.linspace(np.nanmin(T_val),np.nanmax(T_val),num=2*(len(beta)))
    N_mesh,T_mesh = np.meshgrid(N_grid,T_grid,indexing = 'xy')
    grid_s_c = griddata(np.stack((Nden,T_val),axis=1),Ent,(N_mesh,T_mesh),method = 'cubic')
    
    cm = matplotlib.colormaps.get_cmap('seismic')

    # get index of max value for each row
    #idx = np.argmax(grid_s_c, axis=1)
    # use idx to get the x-axis values from T that correspond to max XA
    #x = np.take_along_axis(N_grid, np.expand_dims(idx, axis=-1), axis=-1).squeeze(axis=-1)
    
    #use idx to get the max y-axis values from XA
    #y = np.take_along_axis(T_grid, np.expand_dims(idx, axis=-1), axis=-1).squeeze(axis=-1)


    plt.figure()
#plt.contourf(U_val,T_val,dK_dT)
    plt.figure(figsize = (25,20))
    #plt.title(r"$\frac{\partial \kappa}{\partial T}, N = %sx%s, n =%s$" %(N,N,"1.00"),fontsize = 40)
    ax = plt.gca()
    #plt.xlabel(r"$U$",fontsize = 40)
    #plt.ylabel(r"$T$",fontsize = 40)
    plt.xticks([1.0,1.10,1.20,1.30,1.40],fontsize = 80)
    plt.yticks(fontsize = 80)
    im = plt.contourf(N_mesh,T_mesh,grid_s_c,np.arange(-0.5, 0.5, .01),extend='both',cmap = cm)
    #plt.plot(max_den,Temp,color='green',linestyle = 'dashed',linewidth = 5)
    plt.xlim(1.0,1.4)
    plt.gca().set_ylim(bottom=0.5,top=1.0)
    plt.grid('True',which = 'both')
    CS_2 = ax.contour(N_mesh,T_mesh,grid_s_c, [0.0], colors = ['black'], linestyles = ['dashdot'],linewidths = [5])
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = plt.colorbar(im, cax= cax)
    cbar.ax.tick_params(labelsize = 60)
    plt.tight_layout()
    plt.savefig("%s/Entropy_N_%s_U_%s_contour.png"%(Graph_dir_seebeck,N,u))
    dat0= CS_2.allsegs
    print("Entropy_zeros",dat0)

In [6]:
def entropy_lineplots_T_comp(Text_dir_entropy,Graph_dir,N,U,beta):

            
    plt.figure(figsize = (25,20))
    plt.xticks(fontsize = 80)
    plt.yticks(fontsize = 80)
    #plt.ylim(-2,2)
    
    color_1 = iter(cm.jet(np.linspace(0, 1, int(len(beta)/2)+1)))
    for k1 in range(0,len(beta),2):
        c_1 = next(color_1)
        T_val = 1/(float(beta[k1]))
        filename_entropy = "%s/Text_files_N_%s_U_%s/Thermodynamic_entropy_N_%s_U_%s_beta_%s.dat"%(Text_dir_entropy,N,U,N,U,beta[k1])
        mu_val,num_den,ent,ent_std = np.loadtxt(filename_entropy,unpack = 'True',usecols = [0,1,3,4])
        plt.errorbar(num_den,ent,yerr=ent_std,c=c_1,marker = "o",markersize = 20,elinewidth=3,capsize=5,label = "T=%s"%(str(round(T_val,3))))
    plt.grid("True",which='both')
    plt.legend(loc='lower right',ncol=4,fontsize=50)
    plt.xlim(1,2.0)
    plt.axhline(y = 0,color = 'black',linewidth = 2,linestyle = 'dashed')
    plt.tight_layout()
    plt.savefig("%s/Thermodynamic_entropy_vs_density_N_%s_U_%s.png"%(Graph_dir,N,U))
    plt.close('all')


    plt.figure(figsize = (25,20))
    plt.xticks(fontsize = 80)
    plt.yticks(fontsize = 80)
    #plt.xlabel(r"n",fontsize = 40)
    #plt.ylabel(r"$S_{kelvin}$",fontsize = 40)
    #plt.ylim(-2,2)
    
    color_1 = iter(cm.jet(np.linspace(0, 1, int(len(beta)/2)+1)))
    for k2 in range(0,len(beta),2):
        c_1 = next(color_1)
        T_val = 1/(float(beta[k2]))
        filename_entropy = "%s/Text_files_N_%s_U_%s/Thermodynamic_entropy_N_%s_U_%s_beta_%s.dat"%(Text_dir_entropy,N,U,N,U,beta[k2])
        mu_val,num_den,ent,ent_std = np.loadtxt(filename_entropy,unpack = 'True',usecols = [0,1,3,4])
        plt.errorbar(mu_val,ent,yerr=ent_std,c=c_1,marker = "o",markersize = 20,elinewidth=3,capsize=5,label = "T=%s"%(str(round(T_val,3))))
    plt.grid("True",which='both')
    plt.legend(loc='lower right',ncol=1,fontsize=60)
    #plt.xlim(-7.5,7.5)
    plt.axhline(y = 0,color = 'black',linewidth = 2,linestyle = 'dashed')
    plt.tight_layout()
    plt.savefig("%s/Thermodynamic_entropy_vs_chemical_potential_N_%s_U_%s.png"%(Graph_dir,N,U))
    plt.close('all')


In [34]:
def entropy_lineplots_U_comp(Text_dir_entropy,Graph_dir,N,U,beta):

        
    
    plt.figure(figsize = (25,20))
    plt.xticks(fontsize = 80)
    plt.yticks(fontsize = 80)
    plt.xlim(1.0,1.5)    
    plt.ylim(0.6,0.8)
    color_1 = iter(['olive','cyan','green','purple','blue','orange','red','brown'])
    for k in range(len(U)):

        filename_entropy = "%s/Text_files_N_%s_U_%s/Thermodynamic_entropy_N_%s_U_%s_beta_%s.dat"%(Text_dir_entropy,N,U[k],N,U[k],beta)
        nden,ent,ent_std = np.loadtxt(filename_entropy,unpack = 'True',usecols = [1,3,4])
        c_1 = next(color_1)
        plt.errorbar(nden,ent,yerr=ent_std,c=c_1,marker = "o",markersize = 20,linewidth = 3,elinewidth=3,capsize=10,label = "U=%s"%U[k])

    plt.grid("True",which='both')
    plt.legend(loc='lower left',ncol=3,fontsize=60)

    #plt.ylim(-5.2,5.2)
    plt.axhline(y = 0,color = 'black',linewidth = 2,linestyle = 'dashed')
    plt.tight_layout()
    plt.savefig("%s/Thermodynamic_entropy_vs_density_N_%s_beta_%s.png"%(Graph_dir,N,beta))
    plt.close('all')


    plt.figure(figsize = (25,20))
    plt.xticks(fontsize = 80)
    plt.yticks(fontsize = 80)
    
    color_1 = iter(['olive','cyan','green','purple','blue','orange','red','brown'])
    for k in range(len(U)):

        filename_entropy = "%s/Text_files_N_%s_U_%s/Thermodynamic_entropy_N_%s_U_%s_beta_%s.dat"%(Text_dir_entropy,N,U[k],N,U[k],beta)
        mu_val,ent,ent_std = np.loadtxt(filename_entropy,unpack = 'True',usecols = [0,3,4])
        c_1 = next(color_1)
        plt.errorbar(mu_val,ent,yerr=ent_std,c=c_1,marker = "o",markersize = 20,linewidth = 3,elinewidth=3,capsize=10,label = "U=%s"%U[k])

    plt.grid("True",which='both')
    plt.legend(loc='lower center',ncol=int(len(U)/2),fontsize=50)
    plt.xlim(-4,4)
    plt.ylim(0.5,0.8)
    plt.axhline(y = 0,color = 'black',linewidth = 2,linestyle = 'dashed')
    plt.tight_layout()
    plt.savefig("%s/Thermodynamic_entropy_vs_chemical_potential_N_%s_beta_%s.png"%(Graph_dir,N,beta))
    plt.close('all')


In [35]:
def main():
    
    N = "10"
    dtau = "0.05"
    U = ["3.20","3.40","3.80","4.10","4.50","6.00","8.00","10.00"]
    Trot = ["10","12","14","16","18","20","22","24","26","28","30","32","34","36","38","40","42"]
    Beta = ["2.00","1.50","1.00"] #"1.40","1.30","1.20","1.10","1.00","0.90","0.80","0.70","0.60","0.55","0.50","0.45","0.40","0.35"] #,"0.30"]

    
    Text_dir_main = os.getcwd()
    Text_dir_entropy = "%s/Text_files_N_%s_thermodynamic_entropy_seebeck"%(Text_dir_main,N)
    Graph_dir_entropy = "../Graphs/Graphs_N_%s_entropy"%N

    if not os.path.exists(Graph_dir_entropy):
       os.makedirs(Graph_dir_entropy)
    
    #for k in range(len(U)):
    #   entropy_lineplots_T_comp(Text_dir_entropy,Graph_dir_entropy,N,U[k],Beta)

    for i in range(len(Beta)):  
        entropy_lineplots_U_comp(Text_dir_entropy,Graph_dir_entropy,N,U,Beta[i])



In [36]:
main()